In [37]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords

stopWords = set(stopwords.words('english'))


df = pd.read_csv('scrapped_jobs.csv')

In [38]:
df.rename(columns = {'Location':'Country'}, inplace = True)
df.rename(columns = {'Place':'Location'}, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3150 entries, 0 to 3149
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Keyword          3150 non-null   object 
 1   Country          3150 non-null   object 
 2   Page             3150 non-null   int64  
 3   company          3148 non-null   object 
 4   rating           3095 non-null   float64
 5   Job_title        3150 non-null   object 
 6   Location         3150 non-null   object 
 7   salary           2650 non-null   object 
 8   post_date        3150 non-null   object 
 9   Job_description  3107 non-null   object 
dtypes: float64(1), int64(1), object(8)
memory usage: 246.2+ KB


In [39]:
df = df[df["salary"].str.contains("PerHour") == False]
df = df.dropna()

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2032 entries, 1 to 3148
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Keyword          2032 non-null   object 
 1   Country          2032 non-null   object 
 2   Page             2032 non-null   int64  
 3   company          2032 non-null   object 
 4   rating           2032 non-null   float64
 5   Job_title        2032 non-null   object 
 6   Location         2032 non-null   object 
 7   salary           2032 non-null   object 
 8   post_date        2032 non-null   object 
 9   Job_description  2032 non-null   object 
dtypes: float64(1), int64(1), object(8)
memory usage: 174.6+ KB


In [40]:
df['glassdoor_est'] = df['salary'].apply(lambda x: 1 if '(Glassdoorest.)' in x else 0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2032 entries, 1 to 3148
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Keyword          2032 non-null   object 
 1   Country          2032 non-null   object 
 2   Page             2032 non-null   int64  
 3   company          2032 non-null   object 
 4   rating           2032 non-null   float64
 5   Job_title        2032 non-null   object 
 6   Location         2032 non-null   object 
 7   salary           2032 non-null   object 
 8   post_date        2032 non-null   object 
 9   Job_description  2032 non-null   object 
 10  glassdoor_est    2032 non-null   int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 190.5+ KB


In [41]:
df.head()

,Keyword,Country,Page,company,rating,Job_title,Location,salary,post_date,Job_description,glassdoor_est
1,Bi developer,United States,1,MCKESSON,3.7,BI Developer,"Chantilly, VA",$82K-$109K(Glassdoorest.),3d,McKesson Corporation is a global leader in hea...,1
4,Bi developer,United States,1,Fidelity TalentSource,4.1,BI/WebFOCUS Developer,"New York, NY",$76K-$116K(Glassdoorest.),24h,BI/WebFOCUS Developer\n\nWe are currently sour...,1
6,Bi developer,United States,1,Assigncorp,130.0,Sr. Power BI Developer ( GC/Citizens),"Cambridge, MA",$130K-$140K(Employerest.),15d,We're looking for a team member that enjoys br...,0
7,Bi developer,United States,1,Air Products,4.0,BI Qlik Sense Developer,"Bedford, MA",$68K-$96K(Glassdoorest.),10d,Job Description and Qualifications\n\nAir Prod...,1
9,Bi developer,United States,1,Republic Services,3.5,BI Developer II,"Chicago, IL",$71K-$89K(Glassdoorest.),14d,Business Intelligence Developers at Republic S...,1


In [42]:
salary = df['salary'].apply(lambda x: x.split('(')[0])
minus_Kd = salary.apply(lambda x: x.replace('K','').replace('$',''))
df['min_salary'] = minus_Kd.apply(lambda x: int(float(x.split('-')[0])))
df['max_salary'] = minus_Kd.apply(lambda x: int(float(x.split('-')[-1])))
df['avg_salary'] = (df.min_salary+df.max_salary)/2
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2032 entries, 1 to 3148
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Keyword          2032 non-null   object 
 1   Country          2032 non-null   object 
 2   Page             2032 non-null   int64  
 3   company          2032 non-null   object 
 4   rating           2032 non-null   float64
 5   Job_title        2032 non-null   object 
 6   Location         2032 non-null   object 
 7   salary           2032 non-null   object 
 8   post_date        2032 non-null   object 
 9   Job_description  2032 non-null   object 
 10  glassdoor_est    2032 non-null   int64  
 11  min_salary       2032 non-null   int64  
 12  max_salary       2032 non-null   int64  
 13  avg_salary       2032 non-null   float64
dtypes: float64(2), int64(4), object(8)
memory usage: 238.1+ KB


In [43]:
df["max_salary"].head()

1    109
4    116
6    140
7     96
9     89
Name: max_salary, dtype: int64

In [44]:
df['company'] = df.apply(lambda x: x['company'] if x['rating'] <0 else x['company'].split('\n')[0], axis = 1)


In [45]:
df['company'].head()

1                 MCKESSON
4    Fidelity TalentSource
6               Assigncorp
7             Air Products
9        Republic Services
Name: company, dtype: object

In [46]:
# Create City & State columns
df['City'] = df['Location'].apply(lambda x: x.split(',')[0] if ',' in x.lower() else x)
df['State'] = df['Location'].apply(lambda x: x.split(',')[1] if ',' in x.lower() else x) 
df = df.drop(['Location'],1)

C:\Users\sanand30\AppData\Local\Temp/ipykernel_41324/2703698173.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop(['Location'],1)


In [47]:
df =df.drop(['salary'],1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2032 entries, 1 to 3148
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Keyword          2032 non-null   object 
 1   Country          2032 non-null   object 
 2   Page             2032 non-null   int64  
 3   company          2032 non-null   object 
 4   rating           2032 non-null   float64
 5   Job_title        2032 non-null   object 
 6   post_date        2032 non-null   object 
 7   Job_description  2032 non-null   object 
 8   glassdoor_est    2032 non-null   int64  
 9   min_salary       2032 non-null   int64  
 10  max_salary       2032 non-null   int64  
 11  avg_salary       2032 non-null   float64
 12  City             2032 non-null   object 
 13  State            2032 non-null   object 
dtypes: float64(2), int64(4), object(8)
memory usage: 238.1+ KB


C:\Users\sanand30\AppData\Local\Temp/ipykernel_41324/2156088451.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df =df.drop(['salary'],1)


In [48]:
# Creating a function to encapsulate preprocessing job titles & descriptions, by removing special characters and stop words
def processing(df):
    df['Job_title'] = df['Job_title'].apply(lambda x: ' '.join(x.split()))
    df['Job_title'] = df['Job_title'].apply(lambda x: re.sub(r'[^\w\s]',' ', x.lower()))
    df['Job_description'] = df['Job_description'].apply(lambda x: ' '.join(x.split()))
    df['Job_description'] = df['Job_description'].apply(lambda x: re.sub(r'[^\w\s]','', x.lower()))
    df['Job_description'] = df['Job_description'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopWords]))                                               
    return(df)

In [49]:
# Classifying job titles into each category
def title_simplifier(title):
    if 'data scientist' in title.lower():
        return 'data scientist'
    elif 'data engineer' in title.lower():
        return 'data engineer'
    elif 'analyst' in title.lower():
        return 'data analyst'
    elif 'machine learning' in title.lower():
        return 'machine learning engineer'
    elif 'manager' in title.lower():
        return 'manager'
    elif 'director' in title.lower():
        return 'director'
    else:
        return 'data science related jobs'

In [50]:
# Classifying job levels into each category
def seniority(title):
    if 'sr' in title.lower() or 'senior' in title.lower() or 'vp' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower()or 'experienced' in title.lower() or 'iii' in title.lower() or 'research' in title.lower() or 'manager' in title.lower():
        return 'Senior'
    elif 'intermediate' in title.lower() or 'staff' in title.lower() or 'ii' in title.lower():
        return 'Mid'
    elif 'jr' in title.lower() or 'junior' in title.lower() or 'intern' in title.lower() or 'student' in title.lower()or 'associate' in title.lower():
        return 'Junior'
    else:
        return 'Not Specified'

In [51]:
# Process data based on above defined functions 
df = processing(df)
df['Job'] = df['Job_title'].apply(title_simplifier)
df['Seniority'] = df['Job_title'].apply(seniority)

In [52]:
df.head()

,Keyword,Country,Page,company,rating,Job_title,post_date,Job_description,glassdoor_est,min_salary,max_salary,avg_salary,City,State,Job,Seniority
1,Bi developer,United States,1,MCKESSON,3.7,bi developer,3d,mckesson corporation global leader healthcare ...,1,82,109,95.5,Chantilly,VA,data science related jobs,Not Specified
4,Bi developer,United States,1,Fidelity TalentSource,4.1,bi webfocus developer,24h,biwebfocus developer currently sourcing biwebf...,1,76,116,96.0,New York,NY,data science related jobs,Not Specified
6,Bi developer,United States,1,Assigncorp,130.0,sr power bi developer gc citizens,15d,looking team member enjoys bringing new ideas ...,0,130,140,135.0,Cambridge,MA,data science related jobs,Senior
7,Bi developer,United States,1,Air Products,4.0,bi qlik sense developer,10d,job description qualifications air products ny...,1,68,96,82.0,Bedford,MA,data science related jobs,Not Specified
9,Bi developer,United States,1,Republic Services,3.5,bi developer ii,14d,business intelligence developers republic serv...,1,71,89,80.0,Chicago,IL,data science related jobs,Mid


In [53]:
# Python
df['Python'] = df['Job_description'].apply(lambda x: 1 if 'python' in x.lower() else 0)
# df.Python = df.Python.astype('bool')

# R studio 
df['R'] = df['Job_description'].apply(lambda x: 1 if 'r studio' in x.lower() or 'r-studio' in x.lower() else 0)
# df.R = df.R.astype('bool')

# SQL 
df['SQL'] = df['Job_description'].apply(lambda x: 1 if 'sql' in x.lower() else 0)
# df.SQL = df.SQL.astype('bool')

# Java 
df['JAVA'] = df['Job_description'].apply(lambda x: 1 if 'java' in x.lower() else 0)
# df.JAVA = df.JAVA.astype('bool')

# C++ 
df['C++'] = df['Job_description'].apply(lambda x: 1 if 'c++' in x.lower() else 0)
# df['C++'] = df['C++'].astype('bool')

# AWS 
df['AWS'] = df['Job_description'].apply(lambda x: 1 if 'aws' in x.lower() else 0)
# df.AWS = df.AWS.astype('bool')

# Excel
df['Excel'] = df['Job_description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)
# df.Excel = df.Excel.astype('bool')

# Google Cloud
df['GCP'] = df['Job_description'].apply(lambda x: 1 if 'google cloud' in x.lower() or 'gcp' in x.lower() else 0)
# df.GCP = df.GCP.astype('bool')

# Microsoft Azure
df['Azure'] = df['Job_description'].apply(lambda x: 1 if 'microsoft azure' in x.lower() or 'azure' in x.lower() else 0)
# df.Azure = df.Azure.astype('bool')

# Spark
df['Spark'] = df['Job_description'].apply(lambda x: 1 if 'spark' in x.lower() else 0)
# df.Spark = df.Spark.astype('bool')

# PyTorch
df['PyTorch'] = df['Job_description'].apply(lambda x: 1 if 'pytorch' in x.lower() else 0)
# df.PyTorch = df.PyTorch.astype('bool')

# TensorFlow
df['TensorFlow'] = df['Job_description'].apply(lambda x: 1 if 'tensorflow' in x.lower() or 'tf' in x.lower() else 0)
# df.TensorFlow = df.TensorFlow.astype('bool')

# Tableau
df['Tableau'] = df['Job_description'].apply(lambda x: 1 if 'tableau' in x.lower() else 0)
# df.Tableau = df.Tableau.astype('bool')

# Power BI
df['power_bi'] = df['Job_description'].apply(lambda x: 1 if 'power bi' in x.lower() else 0)
# df.power_bi = df.power_bi.astype('bool')

# Keras
df['Keras'] = df['Job_description'].apply(lambda x: 1 if 'keras' in x.lower() else 0)
# df.Keras = df.Keras.astype('bool')

In [54]:
df = df.drop(['post_date'],1)
df = df.drop(['Job_description'],1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2032 entries, 1 to 3148
Data columns (total 29 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Keyword        2032 non-null   object 
 1   Country        2032 non-null   object 
 2   Page           2032 non-null   int64  
 3   company        2032 non-null   object 
 4   rating         2032 non-null   float64
 5   Job_title      2032 non-null   object 
 6   glassdoor_est  2032 non-null   int64  
 7   min_salary     2032 non-null   int64  
 8   max_salary     2032 non-null   int64  
 9   avg_salary     2032 non-null   float64
 10  City           2032 non-null   object 
 11  State          2032 non-null   object 
 12  Job            2032 non-null   object 
 13  Seniority      2032 non-null   object 
 14  Python         2032 non-null   int64  
 15  R              2032 non-null   int64  
 16  SQL            2032 non-null   int64  
 17  JAVA           2032 non-null   int64  
 18  C++     

C:\Users\sanand30\AppData\Local\Temp/ipykernel_41324/3308247119.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop(['post_date'],1)
C:\Users\sanand30\AppData\Local\Temp/ipykernel_41324/3308247119.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop(['Job_description'],1)


In [56]:
df.to_csv('scraped_jobs_cleaned1.csv', mode = 'a', index =False)

In [ ]:
if SCHEMA_ID('Salary_prediction_database') is NULL
exec ('CREATE SCHEMA salary_prediction_database')

CREATE TABLE salary_prediction_database.glassdoor
(
    Keyword VARCHAR(8000),
    Country VARCHAR(8000),
    page_number int,
    company VARCHAR(8000),
	rating decimal(10,5),
    job_title VARCHAR(8000),
    glassdoor_est bit,
	min_salary decimal(10,4),
	max_salary decimal(10,4),
	avg_salary decimal(10,4),
    city VARCHAR(8000),
	state_s VARCHAR(8000),
    job_t VARCHAR(8000),
    seniority VARCHAR(8000),
    python bit,
	R bit,
	SQL_skill bit,
	AWS bit,
	Excel bit,
	GCP bit,
	Azure bit,
	Spark bit,
	PyTorch bit,
	TensorFlow bit,
	Tableau bit,
	power_bi bit,
	keras bit,
	java bit,
	c_skill bit,
)